In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

# Доп функции

In [610]:
def binary_search_first_occurrence(arr, x):
    left = 0
    right = len(arr) - 1
    result = -1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == x:
            result = mid
            right = mid - 1
            while right >= left and arr[right] == x:
                result = right
                right -= 1
            break
        elif arr[mid] < x:
            left = mid + 1
        else:
            right = mid - 1
    return result

def binary_search_last_occurrence(arr, x):
    left = 0
    right = len(arr) - 1
    result = -1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == x:
            result = mid
            left = mid + 1
            while left <= right and arr[left] == x:
                result = left
                left += 1
            break
        elif arr[mid] < x:
            left = mid + 1
        else:
            right = mid - 1
    return result

# Загрузка данных

In [6]:
train_ds = pd.read_csv('train_dataset_VK/train.csv')

In [7]:
test_ds = pd.read_csv('train_dataset_VK/test.csv')

In [8]:
subm_ds = pd.read_csv('train_dataset_VK/submission.csv')

In [30]:
# test_ego_ids = test_ds.ego_id.unique()

In [31]:
# train_ego_ids = train_ds.ego_id.unique()

In [9]:
attr_ds = pd.read_csv('train_dataset_VK/attr.csv')

In [13]:
attr_ds.head(10)

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1
5,0,284,30,515825832,2,523273865,219762299
6,0,39,107,-1,2,-1,-1
7,0,204,21,298413605,1,-1,70724144
8,0,224,21,298413605,1,-1,-1
9,0,138,43,238321946,1,551175147,359213661


In [18]:
test_ds.head(15)

,ego_id,u,v,t,x1,x2,x3
0,8,20,19,185.7,3.839089e-04,0.0,0.0
1,8,131,125,161.4,4.034464e-01,0.0,0.0
2,8,73,56,127.0,8.554643e-05,0.0,0.0
3,8,0,4,594.5,2.886418e-01,0.0,0.0
4,8,63,73,127.0,4.281692e-07,0.0,0.0
5,8,144,61,217.9,2.454207e+00,0.0,1.0
6,8,0,23,441.7,7.052106e-13,0.0,0.0
7,8,156,125,192.9,8.842460e-11,0.0,0.0
8,8,59,54,118.2,1.513186e+00,0.0,0.0
9,8,109,154,NaN,NaN,0.0,1.0


In [ ]:
test_ds_ego = test_ds['ego_id']

test_dif = test_ds_ego.iloc[1:].values - test_ds_ego.iloc[:-1].values
(test_dif >= 0).all()

In [74]:
train_ds_ego = train_ds['ego_id']

train_dif = train_ds_ego.iloc[1:].values - train_ds_ego.iloc[:-1].values
(train_dif >= 0).all()

True

In [78]:
attr_ds_ego = attr_ds['ego_id']

attr_dif = attr_ds_ego.iloc[1:].values - attr_ds_ego.iloc[:-1].values
(attr_dif >= 0).all()

True

# Попытка в catboost

## Глобальные фичи эго-графа

дополняем train_ds

In [295]:
uniq_ego_ids = set(train_ds['ego_id'].unique())
uniq_ego_ids.update(test_ds['ego_id'].unique())
uniq_ego_ids = sorted(list(uniq_ego_ids))

какие общие свойства могут быть у вершин одного эго-графа?
1) распределение по полу 
2) распр по возрасту
3) топ городов, школ, универов
4) диаметр графа ?
5) ...

будем хранить всё это в ego_ds

In [583]:
# ego_df_cols = ['ego_id', 'age_mean', 'age_std','city_mean', 'city_std', 'sex_mean', 'sex_std', 'sch_mean', 'sch_std', 'univ_mean', 'univ_std']
ego_df_cols = ['ego_id', 'age_mean', 'city_mean', 'sex_mean', 'sch_mean', 'univ_mean']

In [584]:
ego_df = pd.DataFrame(  np.zeros( (len(uniq_ego_ids), len(ego_df_cols))), columns = ego_df_cols)

In [585]:
ego_df['ego_id'] = ego_df['ego_id'].astype('int64')

нужно чем-то заполнить -1 в аттрибутах...


In [587]:
# attr_ds[(0 < attr_ds['sex']) & (attr_ds['sex'] < 100) & (attr_ds['ego_id'] == 12)]['sex'].std()
# attr_ds[(0 < attr_ds['university']) & (attr_ds['ego_id'] == 0)]['university'].mode()[0]

In [589]:
attr_ds[attr_ds < 0] = None

In [591]:
# attr_ds[['ego_id','age','sex','city_id','school','university']].groupby(['ego_id']).mean()

In [593]:
ego_df[['age_mean','sex_mean','city_mean','sch_mean','univ_mean']] = attr_ds[['ego_id','age','sex','city_id','school','university']].groupby(['ego_id']).mean().reset_index(drop=True)

In [364]:
# ego_df[['age_std','sex_std','city_std','sch_std','univ_std']] = attr_ds[['ego_id','age','sex','city_id','school','university']].groupby(['ego_id']).std().reset_index(drop=True)

In [595]:
ego_df['ego_id'] = attr_ds['ego_id'].unique()

In [145]:
# for ego_id, row in tqdm(train_ego_df.iterrows()):
#     age_info = attr_ds[(0 < attr_ds['age']) & (attr_ds['ego_id'] == ego_id)]['age']
#     row['age_mean'] = age_info.mean()
#     row['age_std'] = age_info.std()

#     sex_info = attr_ds[(0 < attr_ds['sex']) & (attr_ds['ego_id'] == ego_id)]['sex']
#     row['sex_mean'] = sex_info.mean()
#     row['sex_std'] = sex_info.std()

#     row['city_mode'] = attr_ds[(0 < attr_ds['city_id']) & (attr_ds['ego_id'] == ego_id)]['city_id'].mode()[0]

#     row['sch_mode'] = attr_ds[(0 < attr_ds['school']) & (attr_ds['ego_id'] == ego_id)]['school'].mode()[0]

#     row['univ_mode'] = attr_ds[(0 < attr_ds['university']) & (attr_ds['ego_id'] == ego_id)]['university'].mode()[0]

94it [00:49,  1.89it/s]


KeyboardInterrupt: 

## Индивидуальные фичи ребра

А здесь задаём заполняем фичи, с уникальными значениями для каждого ребра

фичи ребра (u, v)
1) аттрибуты вершины u (age,city,sex,sch,univ)  (+5)
2) аттрибуты вершины v  (+5)
3) средние значения аттрибутов друзей u (age,city,sex,sch,univ)     (+5) 
4) средние значения аттрибутов друзей v (age,city,sex,sch,univ)     (+5) 
5) количество друзей u  (+1)
6) количество друзей v  (+1)
7) число общих друзей   (+1)
8) транзитивная интенсивность взаимодействия Tr_x1, Tr_x2, Tr_x3    (+3)


$Tr\_x[i](u,v) = среднее ( x[i](u,c) * x[i](v,c) )  \ по\ всем\ общим\ друзьям\ с   $

Что лучше - сумма или произведение?? Мне кажется что всё-таки произведение, чтобы лучше отражать транзитивность

### Эксперименты
Экспериментирую с данными, чтобы понять как составлять фичи

In [614]:
curr_ego_id = 0
curr_idx_train = 0
curr_idx_attr = 0
gr_matr = np.zeros((300,300,4))
attr_curr = np.full((300,5), np.nan)
row = train_ds.iloc[curr_idx_attr]

In [564]:
while attr_ds.iloc[curr_idx_attr]['ego_id'] == curr_ego_id:
    row = attr_ds.iloc[curr_idx_attr]
    attr_cur[int(row['u'])] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[2:]))
    curr_idx_attr += 1


In [565]:
train_ds.iloc[2]['t'] >= 0

False

In [566]:
while train_ds.iloc[curr_idx_train]['ego_id'] == curr_ego_id:
    row = train_ds.iloc[curr_idx_train]
    if row['t'] >= 0:
        u = int(row['u'])
        v = int(row['v'])
        gr_matr[u][v] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[3:]))
        gr_matr[v][u] = gr_matr[u][v]
    curr_idx_train += 1

In [567]:
curr_idx_train_2 = 0

In [568]:
train_ds[(train_ds['ego_id'] == 0) & (train_ds['u'] == 7)]

,ego_id,u,v,t,x1,x2,x3
330,0,7,5,594.5,1.781485e+00,0.0,0.0
417,0,7,6,594.5,1.781925e-01,0.0,0.0
818,0,7,0,396.5,1.362285e-09,0.0,0.0
892,0,7,3,594.5,6.362401e-03,0.0,0.0


In [576]:
(gr_matr[88,:,0] > 0).any()

False

In [569]:
# кол-во друзей у 7
_r = gr_matr[7,:,0] > 0
_r = _r.sum()
_r

5

In [274]:
(gr_matr[7,:,0] > 0).sum()

5

In [275]:
# кол-во общих друзей у 7 и 6
_r = (gr_matr[7,:,0] > 0) & (gr_matr[6,:,0] > 0)
_r = _r.sum()
_r

3

In [276]:
# среднее знач t (индекс 0) по соседям
_r = gr_matr[7,:,0][ gr_matr[7,:,0] > 0 ].mean()
_r

554.9

In [277]:
# среднее знач t (индекс 0) по соседям
_r = gr_matr[7,:,0][ gr_matr[7,:,0] > 0 ].mean()
_r

554.9

In [278]:
# среднее знач x1 по соседям
_r = gr_matr[7,:,1][ gr_matr[7,:,0] > 0 ].mean()
_r

0.07950748186778463

In [279]:
# транзитивная инт между 7 и 6
_r = gr_matr[7,:,1]*gr_matr[6,:,1]
_r.mean()

0.0007511784405331213

In [280]:
np.nanmean(gr_matr[7,:,1:4]*gr_matr[6,:,1:4], axis=0)

array([0.00075118, 0.        , 0.        ])

In [281]:
attr_cur[:,0:5][ gr_matr[7,:,0] > 0 ]

array([[4.30000000e+01, 2.37065842e+08, 1.00000000e+00,            nan,
                   nan],
       [3.50000000e+01,            nan, 1.00000000e+00,            nan,
        8.36217893e+08],
       [4.00000000e+01,            nan, 2.00000000e+00, 8.29727092e+08,
        5.41978296e+08],
       [4.90000000e+01, 2.37065842e+08, 1.00000000e+00, 9.29914814e+08,
        9.00704564e+08],
       [3.00000000e+01, 2.37065842e+08, 1.00000000e+00, 2.05429743e+08,
        6.25011670e+07]])

Всё. Хватит экспериментов

In [282]:
# ср знач возраста и ост
np.nanmean(attr_cur[:,0:5][ gr_matr[7,:,0] > 0 ], axis=0)

array([3.94000000e+01, 2.37065842e+08, 1.20000000e+00, 6.55023883e+08,
       5.85350480e+08])

### Формирование индивидуальных фичей

In [607]:
add_cols_names = ['u','v']
add_cols_names.extend(['u_age', 'u_city', 'u_sex', 'u_sch', 'u_univ'])
add_cols_names.extend(['v_age', 'v_city', 'v_sex', 'v_sch', 'v_univ'])
add_cols_names.extend(['u_age_n', 'u_city_n', 'u_sex_n', 'u_sch_n', 'u_univ_n'])
add_cols_names.extend(['v_age_n', 'v_city_n', 'v_sex_n', 'v_sch_n', 'v_univ_n'])
add_cols_names.extend(['u_cnt', 'v_cnt', 'com_cnt'])
add_cols_names.extend(['tr_x1', 'tr_x2', 'tr_x3'])
add_cols_names.extend(ego_df_cols[1:])
add_cols_names.extend(['x1'])

In [615]:
len(add_cols_names)

34

In [616]:
train_ego_notuniq = train_ds['ego_id'].values
train_ego_uniq = np.unique(train_ego_notuniq)
len_train_ego_uniq = len(train_ego_uniq)

In [672]:
# На случай, если хотим тренироваться только на части данных
actual_len_train = int(0.2 * len_train_ego_uniq)
actual_len_train
import random
random_ego_id_arr = sorted(random.sample(train_ego_uniq.tolist(), actual_len_train))

In [684]:
with open('random_ego_id_arr.npy', 'wb') as f:
    np.save(f, random_ego_id_arr)

А теперь используя все эти штуки БУДЕМ ЗАПОЛНЯТЬ ФИЧИ!

НУ ЧЕ НАРОД? ПОГНАЛИ! 

In [679]:
curr_idx_train = 0
curr_idx_attr = 0
curr_idx_train_2 = 0
curr_idx_ego = 0
curr_idx_part = 0

curr_ego_id = int(train_ds.iloc[0][ego_id])

train_len = train_ds.shape[0]
attr_len = attr_ds.shape[0]
ego_len = ego_df.shape[0]

attr_curr = np.zeros((300,5))
gr_matr = np.zeros((300,300,4))

attr_ego_notuniq = attr_ds['ego_id'].values
ego_ego_notuniq = ego_df['ego_id'].values

In [680]:
import os
if not os.path.exists('ftrs'):
    os.mkdir('ftrs')

In [681]:
import warnings

In [682]:
# pbar = tqdm(total=100)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    #while curr_idx_train < train_len:
    for curr_ego_id in tqdm(random_ego_id_arr):

        #curr_ego_id = train_ds.iloc[curr_idx_train]['ego_id']
        
        attr_curr = np.full((300,5), np.nan)
        gr_matr = np.zeros((300,300,4))
        
        curr_idx_train = curr_idx_train + binary_search_first_occurrence(train_ego_notuniq[curr_idx_train:], curr_ego_id)
        curr_idx_train_end = curr_idx_train + binary_search_last_occurrence(train_ego_notuniq[curr_idx_train:], curr_ego_id)
        # print(f'{curr_idx_train} and {curr_idx_train_end}')
        curr_ftrs_len = curr_idx_train_end - curr_idx_train + 1

        train_ds_ftrs = pd.DataFrame(np.zeros((curr_ftrs_len, len(add_cols_names))), columns=add_cols_names)

        # проходимся по train
        for i in range(curr_ftrs_len):
            row = train_ds.iloc[curr_idx_train + i]
            if row['t'] >= 0:
                u = int(row['u'])
                v = int(row['v'])
                gr_matr[u][v] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[3:]))
                gr_matr[v][u] = gr_matr[u][v]

        # в attr есть ego_id из train, поэтому убедиться что рассматриваемыый
        # ego_id в attr_ds равен ego_id из train_ds
        curr_idx_attr = curr_idx_attr + binary_search_first_occurrence(attr_ego_notuniq[curr_idx_attr:], curr_ego_id)
        curr_idx_attr_end = curr_idx_attr + binary_search_last_occurrence(attr_ego_notuniq[curr_idx_attr:], curr_ego_id)

        # проходимся по attr (нужно чтобы ego_id был тот же)
        while curr_idx_attr <= curr_idx_attr_end:
            row = attr_ds.iloc[curr_idx_attr]
            attr_cur[int(row['u'])] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[2:]))
            curr_idx_attr += 1
    
        curr_idx_ego = curr_idx_ego + binary_search_first_occurrence(ego_ego_notuniq[curr_idx_ego:], curr_ego_id)
        global_ftrs = ego_df.iloc[curr_idx_ego][ego_df_cols[1:]].values

        # проходимся по train ещё раз
        for i in range(curr_ftrs_len):
            row = train_ds.iloc[curr_idx_train + i]
            u = int(row['u'])
            v = int(row['v'])
            ftrs_row = np.zeros(len(add_cols_names))
            ftrs_row[0] = u
            ftrs_row[1] = u
            # фичи u и v
            ftrs_row[2:7] = attr_cur[u][0:5]
            ftrs_row[7:12] = attr_cur[v][0:5]
            
            # u
            # ср знач возраста и ост среди друзей
            # и кол-во друзей
            _u = gr_matr[u,:,0] > 0
            if _r.any():
                ftrs_row[12:17] = np.nanmean(attr_cur[:,0:5][ _u ], axis=0)
                ftrs_row[22] = _u.sum()
            else:
                ftrs_row[12:17] = global_ftrs
                ftrs_row[22] = 0
            # v
            _v = gr_matr[v,:,0] > 0
            if _v.any():
                ftrs_row[17:22] = np.nanmean(attr_cur[:,0:5][ _v ], axis=0)
                ftrs_row[23] = _v.sum()
            else:
                ftrs_row[17:22] = global_ftrs
                ftrs_row[23] = 0
            
            
            # кол-во общих друзей
            if (_u & _v).any():
                ftrs_row[24] = (_u & _v).sum()
            else:
                ftrs_row[24] = 0

            # транзитивная интенсивность (x1, x2, x3)
            ftrs_row[25:28] = np.nanmean(gr_matr[u,:,1:4]*gr_matr[v,:,1:4], axis=0)
            # глоб фичи
            ftrs_row[28:33] = global_ftrs
            # target
            ftrs_row[33] = row['x1']

            train_ds_ftrs.iloc[i] = ftrs_row
            
            pbar.update(100/train_len)
        
        train_ds_ftrs.to_csv(f'ftrs/{int(curr_ego_id)}.csv', index=False)
        
        curr_idx_train = curr_idx_train_end + 1

        # if curr_ego_id > 10:
        #     break


 14%|█▎        | 1687/12357 [1:01:17<6:27:38,  2.18s/it] 


KeyboardInterrupt: 

In [683]:
last_ego_from_train = 231928234507

## Формируем фичи для теста

In [686]:
test_ego_notuniq = test_ds['ego_id'].values
test_ego_uniq = np.unique(test_ego_notuniq)
len_test_ego_uniq = len(test_ego_uniq)


In [ ]:
# # На случай, если хотим тренироваться только на части данных
# actual_len_train = int(0.2 * len_train_ego_uniq)
# actual_len_train
# import random
# random_ego_id_arr = sorted(random.sample(train_ego_uniq.tolist(), actual_len_train))


А теперь используя все эти штуки БУДЕМ ЗАПОЛНЯТЬ ФИЧИ!

НУ ЧЕ НАРОД? ПОГНАЛИ! 

In [691]:
curr_idx_test = 0
curr_idx_attr = 0
curr_idx_test_2 = 0
curr_idx_ego = 0
curr_idx_part = 0
curr_idx_subm = 0

curr_ego_id = int(test_ds.iloc[0][ego_id])

subm_len = subm_ds.shape[0]
test_len = test_ds.shape[0]
attr_len = attr_ds.shape[0]
ego_len = ego_df.shape[0]

attr_curr = np.zeros((300,5))
gr_matr = np.zeros((300,300,4))

attr_ego_notuniq = attr_ds['ego_id'].values
ego_ego_notuniq = ego_df['ego_id'].values

print(len_test_ego_uniq)

20596


In [ ]:
# import os
# if not os.path.exists('ftrs_test'):
#     os.mkdir('ftrs_test')

надо submission смотреть. нужно смотреть пары именно оттуда...

ща сделаем

In [688]:
subm_ego_notuniq = subm_ds['ego_id'].values
subm_ego_uniq = np.unique(subm_ego_notuniq)
len_subm_ego_uniq = len(subm_ego_uniq)

In [689]:
subm_ds_ftrs = pd.DataFrame(np.zeros((subm_len, len(add_cols_names))), columns=add_cols_names)

In [692]:
import warnings
# pbar = tqdm(total=100)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    #while curr_idx_train < train_len:
    for curr_ego_id in tqdm(subm_ego_uniq):

        #curr_ego_id = train_ds.iloc[curr_idx_train]['ego_id']
        
        attr_curr = np.full((300,5), np.nan)
        gr_matr = np.zeros((300,300,4))
        
        curr_idx_test = curr_idx_test + binary_search_first_occurrence(test_ego_notuniq[curr_idx_test:], curr_ego_id)
        curr_idx_test_end = curr_idx_test + binary_search_last_occurrence(test_ego_notuniq[curr_idx_test:], curr_ego_id)
        # print(f'{curr_idx_test} and {curr_idx_test_end}')
        curr_ftrs_len = curr_idx_test_end - curr_idx_test + 1

        test_ds_ftrs = pd.DataFrame(np.zeros((curr_ftrs_len, len(add_cols_names))), columns=add_cols_names)

        # проходимся по test
        for i in range(curr_ftrs_len):
            row = train_ds.iloc[curr_idx_test + i]
            if row['t'] >= 0:
                u = int(row['u'])
                v = int(row['v'])
                gr_matr[u][v] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[3:]))
                gr_matr[v][u] = gr_matr[u][v]
        
        curr_idx_attr = curr_idx_attr + binary_search_first_occurrence(attr_ego_notuniq[curr_idx_attr:], curr_ego_id)
        curr_idx_attr_end = curr_idx_attr + binary_search_last_occurrence(attr_ego_notuniq[curr_idx_attr:], curr_ego_id)

        # проходимся по attr (нужно чтобы ego_id был тот же)
        while curr_idx_attr <= curr_idx_attr_end:
            row = attr_ds.iloc[curr_idx_attr]
            attr_cur[int(row['u'])] = list(map(lambda x: x if x >= 0 else np.NaN, row.values[2:]))
            curr_idx_attr += 1
    
        curr_idx_ego = curr_idx_ego + binary_search_first_occurrence(ego_ego_notuniq[curr_idx_ego:], curr_ego_id)
        global_ftrs = ego_df.iloc[curr_idx_ego][ego_df_cols[1:]].values

        curr_idx_subm = curr_idx_subm + binary_search_first_occurrence(subm_ego_notuniq[curr_idx_subm:], curr_ego_id)
        curr_idx_subm_end = curr_idx_subm + binary_search_last_occurrence(subm_ego_notuniq[curr_idx_subm:], curr_ego_id)
        # print(f'{curr_idx_test} and {curr_idx_test_end}')
        curr_ftrs_subm_len = curr_idx_subm_end - curr_idx_subm + 1
        
        # проходимся по subm
        for i in range(curr_ftrs_subm_len):
            row = subm_ds.iloc[curr_idx_subm + i]
            u = int(row['u'])
            v = int(row['v'])
            ftrs_row = np.zeros(len(add_cols_names))
            ftrs_row[0] = u
            ftrs_row[1] = u
            # фичи u и v
            ftrs_row[2:7] = attr_cur[u][0:5]
            ftrs_row[7:12] = attr_cur[v][0:5]
            
            # u
            # ср знач возраста и ост среди друзей
            # и кол-во друзей
            _u = gr_matr[u,:,0] > 0
            if _r.any():
                ftrs_row[12:17] = np.nanmean(attr_cur[:,0:5][ _u ], axis=0)
                ftrs_row[22] = _u.sum()
            else:
                ftrs_row[12:17] = global_ftrs
                ftrs_row[22] = 0
            # v
            _v = gr_matr[v,:,0] > 0
            if _v.any():
                ftrs_row[17:22] = np.nanmean(attr_cur[:,0:5][ _v ], axis=0)
                ftrs_row[23] = _v.sum()
            else:
                ftrs_row[17:22] = global_ftrs
                ftrs_row[23] = 0
            
            
            # кол-во общих друзей
            if (_u & _v).any():
                ftrs_row[24] = (_u & _v).sum()
            else:
                ftrs_row[24] = 0

            # транзитивная интенсивность (x1, x2, x3)
            ftrs_row[25:28] = np.nanmean(gr_matr[u,:,1:4]*gr_matr[v,:,1:4], axis=0)
            # глоб фичи
            ftrs_row[28:33] = global_ftrs
            # target
            ftrs_row[33] = ftrs_row[25]

            subm_ds_ftrs.iloc[curr_idx_subm + i] = ftrs_row
            
            pbar.update(100/len_subm_ego_uniq)
        
        curr_idx_subm = curr_idx_subm_end + 1

        # if curr_ego_id > 10:
        #     break


 51%|█████     | 10549/20586 [1:02:17<1:17:30,  2.16it/s]

In [ ]:
subm_ds_ftrs.to_csv(f'subm_ftrs.csv', index=False)

## Предсказываем

In [ ]:
import catboost
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
import os

# доп функ

In [ ]:
def binary_search_first_occurrence(arr, x):
    left = 0
    right = len(arr) - 1
    result = -1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == x:
            result = mid
            right = mid - 1
            while right >= left and arr[right] == x:
                result = right
                right -= 1
            break
        elif arr[mid] < x:
            left = mid + 1
        else:
            right = mid - 1
    return result

def binary_search_last_occurrence(arr, x):
    left = 0
    right = len(arr) - 1
    result = -1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == x:
            result = mid
            left = mid + 1
            while left <= right and arr[left] == x:
                result = left
                left += 1
            break
        elif arr[mid] < x:
            left = mid + 1
        else:
            right = mid - 1
    return result

# читаем данные

In [ ]:
with open('random_ego_id_arr.npy', 'rb') as f:
    random_ego_id_arr = np.load(f)

In [ ]:
random_ego_id_arr = random_ego_id_arr[:1686]

In [ ]:
full_len = 0

In [ ]:
for i in tqdm(random_ego_id_arr):
    ds = pd.read_csv(f'ftrs/{i}.csv')
    full_len += ds.shape[0]

100%|██████████| 1686/1686 [00:41<00:00, 40.43it/s]


In [ ]:
print(full_len)

3378910


3378910

In [ ]:
ego_df_cols = ['ego_id', 'age_mean', 'city_mean', 'sex_mean', 'sch_mean', 'univ_mean']

In [ ]:
add_cols_names = ['u','v']
add_cols_names.extend(['u_age', 'u_city', 'u_sex', 'u_sch', 'u_univ'])
add_cols_names.extend(['v_age', 'v_city', 'v_sex', 'v_sch', 'v_univ'])
add_cols_names.extend(['u_age_n', 'u_city_n', 'u_sex_n', 'u_sch_n', 'u_univ_n'])
add_cols_names.extend(['v_age_n', 'v_city_n', 'v_sex_n', 'v_sch_n', 'v_univ_n'])
add_cols_names.extend(['u_cnt', 'v_cnt', 'com_cnt'])
add_cols_names.extend(['tr_x1', 'tr_x2', 'tr_x3'])
add_cols_names.extend(ego_df_cols[1:])
add_cols_names.extend(['x1'])

In [ ]:
part_train_ds = pd.DataFrame(np.zeros((full_len, len(add_cols_names))), columns=add_cols_names)

In [ ]:
last_row_idx = 0

In [ ]:
for ego_id in tqdm(random_ego_id_arr):
    ds = pd.read_csv(f'ftrs/{ego_id}.csv')
    part_train_ds.iloc[last_row_idx:last_row_idx+ds.shape[0]] = ds
    last_row_idx += ds.shape[0]
    

100%|██████████| 1686/1686 [01:24<00:00, 20.05it/s]


In [ ]:
part_train_ds = part_train_ds.fillna(part_train_ds.mean())

In [ ]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

In [ ]:
X, y = part_train_ds[add_cols_names[2:-1]], part_train_ds[add_cols_names[-1]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE', depth=6)

In [ ]:
# grid = {'iterations': [100, 150, 200],
#         'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 8],
#         'l2_leaf_reg': [0.2, 0.5, 1, 3]}
# model.grid_search(grid, train_dataset)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 1.15
R2: 0.25


In [ ]:
subm_ds_ftrs = pd.read_csv(f'subm_ftrs.csv')

In [ ]:
X_subm = subm_ds_ftrs[add_cols_names[2:-1]]

In [ ]:
subm_pred = model.predict(X_subm)

In [ ]:
subm_ds = pd.read_csv('train_dataset_VK/submission.csv')

In [ ]:
subm_ds['x1'] = subm_pred

In [ ]:
subm_ds.to_csv(f'subm_answer.csv', index=False)

,ego_id,u,v,x1
0,8,0,93,0.0
1,8,0,143,0.0
2,8,0,151,0.0
3,8,1,24,0.0
4,8,5,4,0.0
...,...,...,...,...
810971,1709396984676,73,23,0.0
810972,1709396984676,74,68,0.0
810973,1709396984676,77,28,0.0
810974,1709396984676,79,38,0.0
